# Proyecto
______________

API: https://dev.adalab.es/cinema/
Endpoint : https://dev.adalab.es/api/cinema/movies?year=2010&genre=Drama

In [6]:
#Importar librerías para su posterior uso en el código
import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import json
from time import sleep
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Importar librerías de Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [10]:

# Lista de años buscados: 15 años
anio_lista = list(range(2009, 2025))

# Lista de géneros
generos_lista = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

# Lista vacía en la cual almacenaremos los datos extrañidos de la API
datos_peliculas = []

# Bucle para iterar por año y género
for anio in anio_lista:
    for genero in generos_lista:
        # Determinar URL
        url_movies = f"https://dev.adalab.es/api/cinema/movies?year={anio}&genre={genero}"
        
        # Llamada a API
        res_url_movies = requests.get(url_movies)
        
        # Pausa para la API
        sleep(1) 
        
        # Verificar la espuesta de la API
        if res_url_movies.status_code == 200:
            try:
                json_url_general = res_url_movies.json()
                
                # Imprimir jason para verlo
                print(f"Response for {anio} - {genero}: {json_url_general}")
                
                # Verificar si la llamada tiene la info correcta
                if isinstance(json_url_general, dict) and 'results' in json_url_general:
                    movies = json_url_general['results']
                    
                    # Verificar que es lista
                    if isinstance(movies, list):
                        # Bucle por película para extraer info
                        for pelicula in movies:
                            id = pelicula.get("id")
                            titulo = pelicula.get("title")
                            tipo = pelicula.get("type")
                            anio = pelicula.get("year")
                            genero = pelicula.get("genre")
                            
                            # Añadir info a la lista anteriormente vacía
                            datos_peliculas.append({
                                "id": id,
                                "title": titulo,
                                "type": tipo,
                                "year": anio,
                                "genre": genero
                            })
                    else:
                        print("Expected 'results' to be a list.")
                else:
                    print("Expected JSON structure not found.")
            except json.JSONDecodeError:
                print("Error decoding JSON response.")
        else:
            print(f"Error: {res_url_movies.status_code} - {res_url_movies.reason}")

# Convertir a pandas frame
df_peliculas = pd.DataFrame(datos_peliculas)

# Salvar la busqueda en un documento con formato CSV
df_peliculas.to_csv("movies_last_15_years.csv", index=False)

print("Data extraction complete and saved to movies_last_15_years.csv.")


Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway
Error: 502 - Bad Gateway


In [9]:
print(datos_peliculas)
print(type(datos_peliculas))

NameError: name 'datos_peliculas' is not defined

# Necesitamos la siguiente información: **SELENIUM**

1- Puntuación de IMDB (en caso de que la tenga).

2- Dirección (director/a o directores/as de cada película).

3- Guionistas (de cada película).

4- Argumento.

5- Nombre de la película.

6- Id de la película.

In [8]:
titulo_peli_api = [titulo ["title"]for titulo in datos_peliculas]
print(titulo_peli_api)

NameError: name 'datos_peliculas' is not defined

In [7]:
# Sacar titulos de peliculas y almacenarlo en una variable
titulo_peli_api = [titulo ["title"]for titulo in datos_peliculas]
print(titulo_peli_api)

#Iniciar el navegador
driver = webdriver.Chrome()
print("Abriendo Google")

# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")
sleep(3)

#Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")
sleep(3)

#COOKIES
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")
sleep(5)

datos_imdb = {"puntuacion":[], "direcccion":[], "guionista":[], "argumento":[], "nombre_pelicula":[], "id_pelicula":[]}

for titulo in  titulo_peli_api:
  buscador = driver.find_element("css selector", "#suggestion-search").send_keys(titulo, Keys.ENTER)
  sleep(5)
  try:
    encontrado = driver.find_element("link text",f"{titulo}").click()
    if encontrado == True:
      puntuacion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-3a4309f8-0.jJkxPn.sc-1f50b7c-1.doVJqQ > div > div:nth-child({titulo}) > a > span > div > div.sc-eb51e184-0.ghvwpw > div.sc-eb51e184-2.fAhXAe > span.sc-eb51e184-1.ljxVSS").text
      datos_imdb["puntuacion"].append(puntuacion)
      print(f"datos_imdb{puntuacion}")
      
      direccion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child({titulo}) > div > ul > li > a").text
      datos_imdb["direccion"].append(direccion)
      print(f"datos_imdb{direccion}")

      guionista = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-11.hpMyOS > div.sc-1f50b7c-2.cpicUu > div > ul > li:nth-child({titulo}) > div > ul > li > a").text
      datos_imdb["guionista"].append(guionista)
      print(f"datos_imdb{guionista}")

      argumento = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child({titulo}) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > p > span.sc-2d37a7c7-1.dCcJCA").text
      datos_imdb["argumento"].append(argumento)
      print(f"datos_imdb{argumento}")

      nombre_pelicula = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child({titulo}) > section > section > div.sc-491663c0-3.kNMkxz > div.sc-1f50b7c-0.iPPbjm > h1 > span").text
      datos_imdb["nombre_pelicula"].append(nombre_pelicula)
      print(f"datos_imdb{nombre_pelicula}")

      id_pelicula = driver.current_url.split("/")[4]
      datos_imdb["id_pelicula"].append(id_pelicula) #tt014355
      print(f"datos_imdb{id_pelicula}")
    
    print(datos_imdb)
    
  except:
          continue

Abriendo Google
Pantalla maximizada
Accediendo a la web
Cookies aceptadas


NameError: name 'datos_peliculas' is not defined